## 实战项目：图像标注

---

在这个notebook中，我们要做的是训练自己的循环神经网络模型。


### 任务 #1

首先，请设置以下变量：
- `batch_size` - 每个训练批次的批次大小。它是指用于在每个训练步骤中修改模型权重的图像标注对的数量。
- `vocab_threshold` - 单词阈值最小值。请注意，阈值越大，词汇量越小，而阈值越小，则表示将包括较少的词汇，词汇量则越大。
- `vocab_from_file` - 一个布尔值，用于决定是否从文件加载词汇表。
- `embed_size` - the dimensionality of the image and word embeddings.  图像和单词嵌入的维度。
- `hidden_size` - RNN解码器隐藏状态下的特征数。
- `num_epochs` - 训练模型的epoch数。
- `save_every` - 确定保存模型权重的频率。我们建议你设置为`save_every=1`，便于在每个epoch后保存模型权重。这样，在第`i`个epoch之后，编码器和解码器权重将在`models/`文件夹中分别保存为`encoder-i.pkl`和`decoder-i.pkl`。
- `print_every` - 确定在训练时将批次损失输出到Jupyter notebook的频率。
- `log_file` - 包含每个步骤中训练期间的损失与复杂度演变过程的的文本文件的名称。


### 问题1

**问题:** 详细描述你的CNN-RNN架构。对于这种架构，任务1中变量的值，你是如何选择的？如果你查阅了某一篇详细说明关于成功实现图像标注生成模型的研究论文，请提供该参考论文。

**答案:** 
- 我参考了[ShowandTell: ANeuralImageCaptionGenerator](https://arxiv.org/pdf/1411.4555.pdf)这篇论文,使用LSTM这个来处理这个序列问题。适用dropout层来进一步抑制过拟合。我先把输入数据，使用Embedding层进行词嵌入，将整个描述说明序列的每个单词转换到相同维数，然后将input样本输入到LSTM得到每次处理的隐藏状态，即输出，再传给一个全连接层，输出的个数是我们统计所有单词的个数，因为我们对每个单词的输出做客独热编码。在参考了论文后，我设置batch_size是64，vocab_threshold是5，embed_size取256，hidden_size是512.采取的具体策略是编码器使用的CNN，对图片进行了编码，然后再将CNN处理的图片进行转换，使它的维度和embed_size一样，都是256维度。将这个图片编码的结果作为解码器的第一个输入，它预测的结果是描述序列的第一个词，也就是START字符,，然后第二个输入就是描述序列的一个词，预测结果是序列的第二个词，依次类推。最后的输入是倒数第二个词，预测结果是END字符，表示结束。


### 问题2

**问题:** 你是如何在`transform_train`中选择转换方式的？如果你将转换保留为其提供的值，为什么你的任务它非常适合你的CNN架构？

**答案:** 
- 我使用transform操作是先把图片调整到256大小，因为有的图片大于256，有的图片小于256，我们使用的预训练模型需要224x224的长宽，我先将他们调整到256大小，然后将他们沿着中心裁剪为224X224，再使用一个随机水平反转，确保图片位置的多样性。组后对他们做归一化，就是减去mean再除以std，即转换为标准正态分布。使用CNN是因为卷积网络非常适合对图片进行特征提取，他对图片的通道和空间信息都有很好的过滤，我们使用一个优良的CNN架构，这就保证了模型的深度，使得我们可以对一个图片从边缘识别的简单提取，随着深度的增加可以提取到图片各部位的更加抽象高级的特征，比如人脸，一开始我们这提取轮廓边缘的特征，到后边可以提取到眉毛眼睛等高级特征，而CNN具有的平移不变性符合我们人眼视觉的特点。所以CNN非常适合提取图片特征，用CNN作为我们的解码器是合适的
### 任务 #3

接下来，你需要定义一个包含模型的可学习参数的Python列表。 例如，如果你决定使解码器中的所有权重都是可训练的，但只想在编码器的嵌入层中训练权重，那么，就应该将`params`设置为：

In [ ]:
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

### 问题3

**问题:** 你是如何选择该架构的可训练参数的？ 为什么你认为这是一个不错的选择？

**答案:** 
- 首先解码器阶段，我们使用迁移学习的方式来提取图片特征，也就是使用预训练模型，这极大提升了我们的工作效率，使我们可以省去大部分训练的时间。这种预训练的模型是经过长期验证得到的高效优良模型，我们使用resnet预训练模型非常适合处理图像问题，和我们的需求很相似。我们没必要再重新训练模型。而通过预训练模型得到结果，并不能满足我们的需求，我们需要对其做一个Embedding嵌入操作，使得和我们的描述序列的每个词都是同样的维度吗，因此我们需要一个全连接层将其转换为相同的embed维度。这个全连接层是需要我们训练的。而解码器中是我们自己定义的模型架构，因此需要我们对其相关的所有权重参数进行训练。这样的可以得到一个比较满意的模型


### 问题4

**问题:** 你是如何选择用于训练模型的优化程序的？

**答案:**
首先我们使用dropout，dropout可以按概率随机将部分隐含层节点的权重归零，由于每次迭代受归零影响的节点不同，因此各节点的重要性会被平衡，这其实是个平均化思想，不要把一直放在权重较高的节点上，同时也要照顾到较低的权重。然后使用adam优化器算法，它结合AdaGrad和RMSProp两种优化算法的优点，很适合应用于大规模的数据及参数的场景和适用于梯度稀疏或梯度存在很大噪声的问题。目前基本都是用adam作为默认优化器。

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import torch.optim as optim


batch_size = 64          
vocab_threshold = 5      
vocab_from_file = True   
embed_size = 256    #词嵌入维度       
hidden_size = 512   #状态变量维度
num_epochs = 3             
save_every = 1             
print_every = 100          
log_file = 'training_log.txt' 

#用于数据增强
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          
    transforms.RandomCrop(224),                      
    transforms.RandomHorizontalFlip(),               
    transforms.ToTensor(),                          
    transforms.Normalize((0.485, 0.456, 0.406),      
                         (0.229, 0.224, 0.225))])

data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

#词汇数量
vocab_size = len(data_loader.dataset.vocab)

#初始化编码器和解码器 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

#如果有gpu并且支持cuda，可以使用gpu训练，否则使用cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

#定义损失函数 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

#定义哪些参数需要参与训练.编码器的embed词嵌入层和解码器的所有层参数
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

#定义adam优化器
optimizer = optim.Adam(params, lr=0.001)

total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.51s)
creating index...
index created!
Obtaining caption lengths...


100%|████████████████████████████████████████████████████████████████████████| 414113/414113 [01:18<00:00, 5302.49it/s]
D:\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<a id='step2'></a>
## Step 2: 训练你的模型

在**Step 1**中执行代码单元格后，就可以进行下面的训练模型的阶段了。

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

#记录训练日志
f = open(log_file, 'w')

for epoch in range(1, num_epochs+1):
    h = decoder.init_hidden(64)
    for i_step in range(1, total_step+1):
      
        # 获取序列长度等于某个随机值的样本索引.
        indices = data_loader.dataset.get_train_indices()
        # 根据获取的样本索引进行批量采样.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        #获得批量数据
        images, captions = next(iter(data_loader))

        #我们使用的是gpu训练。将数据交给cuda计算
        images = images.to(device)
        captions = captions.to(device)
        h = tuple([each.data for each in h])
        # 每次获取新的批量数据，要对之前的梯度清0
        decoder.zero_grad()
        encoder.zero_grad()
        
        #将输入数据传给我们的编码器-解码器架构模型来处理.
        features = encoder(images)
        outputs, h = decoder(features, captions,h)
        
        #计算批量损失
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        #进行反向传播.
        loss.backward()
        
        #通过优化器更新参数.
        optimizer.step()
            
        #获取训练统计信息.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        #将信息写入日志文件
        f.write(stats + '\n')
        f.flush()
        
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # 保存权重，每save_every个间隔保存一次权重。分别保存编码器和解码器的权重。下次可以直接加载这些权重
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

f.close()

Epoch [1/3], Step [100/6471], Loss: 4.3183, Perplexity: 75.0597
Epoch [1/3], Step [200/6471], Loss: 3.8799, Perplexity: 48.42068
Epoch [1/3], Step [300/6471], Loss: 3.4509, Perplexity: 31.52904
Epoch [1/3], Step [400/6471], Loss: 3.4717, Perplexity: 32.1924
Epoch [1/3], Step [500/6471], Loss: 3.7226, Perplexity: 41.37222
Epoch [1/3], Step [600/6471], Loss: 3.1691, Perplexity: 23.78584
Epoch [1/3], Step [700/6471], Loss: 3.3633, Perplexity: 28.88402
Epoch [1/3], Step [800/6471], Loss: 3.0314, Perplexity: 20.7269
Epoch [1/3], Step [900/6471], Loss: 3.1502, Perplexity: 23.3405
Epoch [1/3], Step [1000/6471], Loss: 3.2243, Perplexity: 25.1352
Epoch [1/3], Step [1100/6471], Loss: 3.0306, Perplexity: 20.7102
Epoch [1/3], Step [1200/6471], Loss: 2.7712, Perplexity: 15.9782
Epoch [1/3], Step [1300/6471], Loss: 2.7818, Perplexity: 16.1474
Epoch [1/3], Step [1400/6471], Loss: 2.8917, Perplexity: 18.0247
Epoch [1/3], Step [1500/6471], Loss: 3.2716, Perplexity: 26.3545
Epoch [1/3], Step [1600/6471]